In [3]:
# from https://github.com/ChameleonCloud/bits-o-scripts/blob/master/managers.py

In [2]:
class ClientManager(object):
    def __init__(self, rc):
        self.key = self._get_keystone_client(rc)
        self.lease_manager = self._get_lease_manager()
        self.heat = self._get_heat_client(rc)

    def _get_keystone_client(self, rc):
        key = keystoneclient.Client(**{
            'auth_url': rc['OS_AUTH_URL'],
            'tenant_id': rc['OS_TENANT_ID'],
            'tenant_name': rc['OS_TENANT_NAME'],
            'password': rc['OS_PASSWORD'],
            'region_name': rc['OS_REGION_NAME'],
            'username': rc['OS_USERNAME'],
        })
        key.authenticate()
        return key

    def _get_lease_manager(self):
        climate_url = self.key.service_catalog.url_for(service_type='reservation')
        return climateclient.v1.leases.LeaseClientManager(climate_url, self.key.auth_token)

    def _get_heat_client(self, rc):
        os_auth_url = rc['OS_AUTH_URL']
        username = rc['OS_USERNAME']
        password = rc['OS_PASSWORD']
        project = rc['OS_PROJECT_NAME']

        keystone_session = keystoneauth1.session.Session(verify=True, cert="", timeout=None)

        keystone_auth = keystoneauth1.identity.generic.Password(**{
            'username': username,
            'user_id': '',
            'user_domain_id': '',
            'user_domain_name': '',
            'password': password,
            'auth_url': os_auth_url,
            'project_id': '',
            'project_name': project,
            'project_domain_id': '',
            'project_domain_name': '',
        })

        return heatclient.client.Client('1', **{
            'auth_url': os_auth_url,
            'session': keystone_session,
            'auth': keystone_auth,
            'service_type': 'orchestration',
            'endpoint_type': 'publicURL',
            'region_name': '',
            'username': username,
            'password': password,
            'include_pass': False
        })


class InstantStack(object):
    def __init__(self, cm, name, ssh_keyname, nodes=2):
        self.name = name
        self.nodes = nodes
        self.ssh_keyname = ssh_keyname

        self.lease_name = None
        self.lease_id = None
        self.reservation_id = None
        self.stack_name = None
        self.stack_id = None

        self._stack_update = 0
        self._lease_update = 0
        self._stack_refresh = self._lease_refresh = 8 # seconds

        self.cm = cm
        self.heat = cm.heat
        self.lease_manager = cm.lease_manager

    def __repr__(self):
        return '<{}: {}, lease={}, stack={}>'.format(
            self.__class__.__name__, self.name, self.lease_id, self.stack_id)

    @property
    def lease(self):
        now = monotonic()
        if now > self._stack_update:
            self._lease_update = now + self._lease_refresh
            self._lease_dic = self.lease_manager.get(self.lease_id)

        return self._lease_dic

    def all_leases(self):
        return [l for l in self.lease_manager.list() if l['status'] != 'STOP']

    def create_lease(self, lease_name=None, start=None, length=3*60*60, trap=False):
        if start is None:
            start = datetime.datetime.utcnow() + datetime.timedelta(minutes=1, seconds=15)

        if self.lease_name is not None:
            raise RuntimeError('lease already created with name {}'.format(self.lease_name))

        if lease_name is None:
            lease_name = 'mpidemo-lease-{}'.format(self.name)

        end = start + datetime.timedelta(seconds=length)

        FORMAT = '%Y-%m-%d %H:%M'

        payload = {
            'name': lease_name,
            'start': start.strftime(FORMAT),
            'end': end.strftime(FORMAT),
            'events': [],
            'reservations': [{
                'hypervisor_properties': '',
                'max': str(self.nodes),
                'min': str(self.nodes),
                'resource_properties': '',
                'resource_type': 'physical:host',
            }],
        }

        try:
            response = self.lease_manager.create(**payload)

        except Exception as e:
            if not trap:
                raise
            self.lease_id = e

        else:
            self.lease_id = response['id']
            self.lease_name = lease_name
            self.reservation_id = response['reservations'][0]['id']
            self._last_response = response

    def delete_lease(self, lease_name=None):
        if lease_name is None:
            if self.lease_name is None:
                raise RuntimeError('must provide name to delete')
            lease_name = self.lease_name
            lease_id = self.lease_id
        else:
            raise ValueError('dereferencing names TBD.')

        response = self.lease_manager.delete(lease_id)

        self.lease_id = None
        self.lease_name = None

        self._last_response = response

    def all_stacks(self):
        return list(self.heat.stacks.list())

    def create_stack(self, stack_name=None, template=None, trap=False):
        if stack_name is None:
            stack_name = 'mpidemo-stack-{}'.format(self.name)
        if template is None:
            template = TEMPLATE
        if self.reservation_id is None:
            raise RuntimeError('bad state: no reservation_id')

        HTV = 'heat_template_version'
        if isinstance(template[HTV], (datetime.datetime, datetime.date)):
            template[HTV] = template[HTV].strftime('%Y-%m-%d')

        env = {'parameters': {
            'reservation_id': self.reservation_id,
            'node_count': self.nodes,
            'key_name': self.ssh_keyname,
        }}

        try:
            response = self.heat.stacks.create(**{
                'stack_name': stack_name,
                'disable_rollback': True,#not(args.enable_rollback),
                'template': template,
                'environment': env,
            })
        except Exception as e:
            if not trap:
                raise
            self.stack_id = e

        else:
            self.stack_id = response['stack']['id']
            self.stack_name = stack_name
            self._last_response = response

    def delete_stack(self, stack_name=None):
        if stack_name is None:
            if self.stack_name is None:
                raise RuntimeError('must provide name to delete')
            stack_name = self.stack_name
            stack_id = self.stack_id

        self.heat.stacks.delete(stack_id)

        self.stack_id = None
        self.stack_name = None

    @property
    def stack(self):
        now = monotonic()
        if now > self._stack_update:
            self._stack_update = now + self._stack_refresh
            self._stack_obj = self.heat.stacks.get(self.stack_name)

        return self._stack_obj

    @property
    def stack_output(self):
        if self.stack.stack_status == 'CREATE_IN_PROGRESS':
            raise RuntimeError('stack still being created, try later')

        return {out['output_key']: out['output_value'] for out in self.stack.outputs}

    def test_stack(self):
        fabric.api.env['host_string'] = self.stack_output['first_instance_ip']
        fabric.api.env['user'] = 'cc'
        fabric.api.env['disable_known_hosts'] = True

        result = fabric.api.run(
            'mpirun -f hosts /usr/mpi/examples/mpi_helloworld',
            quiet=True,
        )

        assert 'Hello from task 0 on mpi-instance-0!' in result
        assert 'Hello from task 1 on mpi-instance-1!' in result
        assert result.return_code == 0

        return result.return_code